# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# TODO: da rifare per ogni stock
sheet_names = [
    'Info',
    'Historical',
    'Income Statement',
    'Quarterly Income Statement',
    'Cashflow',
    'Institutional Holders',
    'Mutual Fund Holders',
    'Major Holders'
]

#riempire stocks di tutti i vari codici, fare la retrive di tutti i file e buttarli in df_stock per poi poter lavorare su tutti i dati
directory = "./data"
stocks = [os.path.join(directory, file) for file in os.listdir(directory)]
stocks

['./data/GS.xlsx',
 './data/1398.HK.xlsx',
 './data/BA.xlsx',
 './data/005380.KS.xlsx',
 './data/AIR.PA.xlsx',
 './data/KO.xlsx',
 './data/WMT.xlsx',
 './data/005930.KS.xlsx',
 './data/NOVN.SW.xlsx',
 './data/CVX.xlsx',
 './data/EGHT.xlsx',
 './data/MMM.xlsx',
 './data/VZ.xlsx',
 './data/JPM.xlsx',
 './data/JNJ.xlsx',
 './data/AMZN.xlsx',
 './data/9984.T.xlsx',
 './data/RDS.A.xlsx',
 './data/0700.HK.xlsx',
 './data/BE.xlsx',
 './data/AAPL.xlsx',
 './data/PFE.xlsx',
 './data/SEDG.xlsx',
 './data/MSFT.xlsx',
 './data/TM.xlsx',
 './data/VOW3.DE.xlsx',
 './data/PG.xlsx',
 './data/DB.xlsx',
 './data/UL.xlsx',
 './data/TWTR.xlsx',
 './data/XOM.xlsx',
 './data/T.xlsx']

## Integrazione dei dati finanziarii
Colonne aggiunte:
- **Daily_Return**: rendimento giornaliero.
- **Target_1day**: indica se il prezzo di chiusura del giorno successivo sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.
- **Target_5days**: indica se il prezzo di chiusura a 5 giorni nel futuro sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.
- **Target_30days**: indica se il prezzo di chiusura a 30 giorni nel futuro sarà superiore (1) o inferiore (0) rispetto al prezzo di chiusura del giorno corrente.

Integrato i vari sheet "Income Statement", "Quarterly Income Statement" e "Cashflow" in un singolo excel. NB: Dato che questi fogli contengono dati finanziari annuali o trimestrali un approccio comune è portare avanti l'ultimo valore noto per ogni giorno fino a quando non si dispone di un nuovo valore. Per alcunii anni finanziari sarà Nan perché non li abbiamo.

Lista delle azioni alle quali mancano pezzi:
- **1398.HK** manca income_statement.normalized_EBITA

In [17]:
file = './data/XOM.xlsx'
file.split("/")[2].replace(".", "")[:-4]

'XOM'

In [22]:
# TODO: da fare per ogni stock
counter = 0
for file in stocks:
    if file.split("/")[2].replace(".", "")[:-4] != "1398.HK":
        df_stock = pd.ExcelFile(file)
        
        # prevent false postive warnings, reference_ https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
        pd.options.mode.chained_assignment = None # default='warn'
        
        # Loading the 'Historical' data stock
        historical_data = df_stock.parse('Historical')
        
        # Renaming and setting the Date column
        historical_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
        historical_data['Date'] = pd.to_datetime(historical_data['Date'])
        historical_data.set_index('Date', inplace=True)
        # Calculate daily return
        historical_data['Daily_Return'] = historical_data['Close'].pct_change()
        
        # Create target variables for next day, next 5 days and next 30 days
        historical_data['Target_1day'] = (historical_data['Close'].shift(-1) > historical_data['Close']).astype(int)
        historical_data['Target_5days'] = (historical_data['Close'].shift(-5) > historical_data['Close']).astype(int)
        historical_data['Target_30days'] = (historical_data['Close'].shift(-30) > historical_data['Close']).astype(int)
        
        # Drop rows with NaN values (will be present due to the shifting for target creation)
        historical_data = historical_data.dropna()
        
        # Loading the 'Income Statement' data for XOM
        income_statement = df_stock.parse('Income Statement')
        
        # Transposing the data for easier integration
        income_statement = income_statement.set_index('Unnamed: 0').transpose()
        income_statement.index = pd.to_datetime(income_statement.index)
        
        
        # Selecting some of the key financial metrics (you can add or remove based on relevance)
        selected_metrics = [
            'Normalized EBITDA',
            'Total Unusual Items',
            'Total Unusual Items Excluding Goodwill'
        ]
        
        # check if columns exist, in case create them
        for metric in selected_metrics:
            if metric not in income_statement.columns:
                income_statement[metric] = np.nan
                
        
        income_statement = income_statement[selected_metrics]
        
        # Merging the income statement data with the historical data
        merged_data = historical_data.join(income_statement, how='left')
        
        # Forward filling the NaN values
        merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
        
        # Loading the 'Cashflow' data for XOM
        cashflow = df_stock.parse('Cashflow')
        
        # Transposing the data for easier integration
        cashflow = cashflow.set_index('Unnamed: 0').transpose()
        cashflow.index = pd.to_datetime(cashflow.index)
        
        # Selecting some of the key cashflow metrics (you can add or remove based on relevance)
        selected_cashflow_metrics = [
            'Operating Cash Flow',
            'Capital Expenditure',
            'Free Cash Flow'
        ]
        
        for metric in selected_cashflow_metrics:
            if metric not in cashflow.columns:
                cashflow[metric] = np.nan
        
        cashflow = cashflow[selected_cashflow_metrics]
        
        # Merging the cashflow data with the existing dataframe
        merged_data = merged_data.join(cashflow, how='left', rsuffix='_cashflow')
        
        # Forward filling the NaN values
        merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')
        
        if 'Ticker' not in merged_data.columns:
            merged_data['Ticker'] = file.split("/")[2].replace(".", "")[:-4]
        
        # Display the updated dataframe with integrated cashflow metrics
        merged_data.iloc[counter : counter + len(merged_data), merged_data.columns.get_loc("Ticker")] = file.split("/")[2].replace(".", "")[:-4]
        
        counter = len(merged_data)

/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:55: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:81: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')


GS


/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:55: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:81: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')


1398HK


/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:55: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:81: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')


...

/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:55: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:81: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')


TWTR


/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:55: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:81: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')


XOM
T


/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:55: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
/var/folders/kl/fcwqs6xs30b3g73_r8l0_q5m0000gn/T/ipykernel_26216/1327827656.py:81: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')


In [23]:
print(merged_data.Ticker)

Date
1983-11-22    T
1983-11-23    T
1983-11-25    T
1983-11-28    T
1983-11-29    T
             ..
2023-10-09    T
2023-10-10    T
2023-10-11    T
2023-10-12    T
2023-10-13    T
Name: Ticker, Length: 10055, dtype: object


In [ ]:
# TODO: spiegare perchè togliamo i quarterly
#merged_data.drop(columns=['Normalized EBITDA_quarterly', 'Total Unusual Items_quarterly', 'Total Unusual Items Excluding Goodwill_quarterly'], inplace=True)
merged_data.iloc[1 : counter + len(merged_data), merged_data.columns.get_loc("Ticker")] = file.split("/")[1].split("\\")[1].replace(".", "")[:-4]

## Feature Engineering
- **Medie mobili**: Calcoliamo le medie mobili a breve e lungo termine per il prezzo di chiusura, che sono comuni nel trading algoritmico. Ad esempio, medie mobili a 5, 10, 30 e 50 giorni.
- **RSI (Relative Strength Index)**: Questo è un indicatore di momentum che può aiutare a identificare se un'azione è in condizione di "overbought" o "oversold".
- **MACD (Moving Average Convergence Divergence)**: Un altro indicatore di momentum.
- **Bollinger Bands**: Questi sono basati su medie mobili e possono aiutare a identificare se un prezzo è relativamente alto o basso.
- **Volatilità**: Potremmo calcolare la volatilità come la deviazione standard dei rendimenti giornalieri in una finestra temporale specifica.

In [ ]:
# TODO: da fare per ogni stock
# TODO: controllare gpt

# Moving Averages
merged_data['MA_5'] = merged_data['Close'].rolling(window=5).mean()
merged_data['MA_10'] = merged_data['Close'].rolling(window=10).mean()
merged_data['MA_30'] = merged_data['Close'].rolling(window=30).mean()
merged_data['MA_50'] = merged_data['Close'].rolling(window=50).mean()

# RSI
delta = merged_data['Close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
merged_data['RSI'] = 100 - (100 / (1 + rs))

# MACD
merged_data['MACD'] = merged_data['Close'].ewm(span=12, adjust=False).mean() - merged_data['Close'].ewm(span=26, adjust=False).mean()
merged_data['Signal_Line'] = merged_data['MACD'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
merged_data['Bollinger_Mid_Band'] = merged_data['Close'].rolling(window=20).mean()
merged_data['Bollinger_Upper_Band']  = merged_data['Bollinger_Mid_Band'] + 1.96*merged_data['Close'].rolling(window=20).std()
merged_data['Bollinger_Lower_Band']  = merged_data['Bollinger_Mid_Band'] - 1.96*merged_data['Close'].rolling(window=20).std()

# Volatility
merged_data['Volatility'] = merged_data['Daily_Return'].rolling(window=5).std()

to_drop_na = ['MA_5', 'MA_10', 'MA_30', 'MA_50', 'RSI', 'Volatility']

for column in to_drop_na:
    merged_data[column] = merged_data[column].fillna(0)

# Display the dataset with new features
merged_data

In [ ]:
# TODO: spiegare perche tagliamo il numero di record
merged_data = merged_data[merged_data.index >= '2020-06-30']
merged_data

In [ ]:
# da fare per ogni stock
output_filepath = "processed_nomedellostock.xlsx"
len(merged_data)
merged_data.to_excel(output_filepath)
